In [1]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision   # 数据库模块
import matplotlib.pyplot as plt

# Hyper Parameters

In [3]:
EPOCH = 1
BATCH_SIZE = 50
LR = 0.001
DOWNLOAD_MNIST = True # 是否要下载MNIST数据集

# 加载MNIST数据集

In [4]:
train_data = torchvision.datasets.MNIST(
    root='./mnist/',   # 保存数据集的位置
    train=True,      # 加载训练集
    transform=torchvision.transforms.ToTensor(),
    download=DOWNLOAD_MNIST
)

100.1%

Extracting ./mnist/MNIST\raw\train-images-idx3-ubyte.gz to ./mnist/MNIST\raw


113.5%

Extracting ./mnist/MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist/MNIST\raw


100.4%

Extracting ./mnist/MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist/MNIST\raw


180.4%

Extracting ./mnist/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist/MNIST\raw
Processing...
Done!


In [5]:
test_data = torchvision.datasets.MNIST(root='./mnist/', train=False)

In [9]:
# 批训练：50 samples，1 channel，28x28（50，1，28，28）
train_loader = Data.DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# shape from (2000, 28, 28) to (2000, 1, 28, 28), value in range(0,1)
test_x = torch.unsqueeze(test_data.data, dim=1).type(torch.FloatTensor)[:2000]/255
test_y = test_data.targets[:2000]

# 搭建模型

PyTorch中 `view` 的用法:

把原先tensor中的数据按照**行优先**的顺序排成一个一维的数据，然后按照参数指定的shape重构tensor。

参数中的-1代表这个位置由其他位置的数字来推断。

In [12]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Sequential(  # input shape (1,28,28)
            nn.Conv2d(
                in_channels=1,   # input channels
                out_channels=16,  # n_filters
                kernel_size=5,   # filter size
                stride=1,        # filter step
                padding=2
            ),  # output shape (16,28,28)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2) # 在2x2空间里向下采样，output shape(16,14,14)
        )
        
        self.conv2 = nn.Sequential(   # input shape(16,14,14)
            nn.Conv2d(16,32,5,1,2),   # output shape(32,14,14)
            nn.ReLU(),
            nn.MaxPool2d(2) # output shape(32,7,7)
        )
        
        self.out = nn.Linear(32 * 7 * 7, 10) # fully connected layer, output 10 classes
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)  # 展平多维的卷积结果为(batch_size, 32*7*7)
        output = self.out(x)
        return output

In [14]:
cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


# 训练

In [16]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

for epoch in range(EPOCH):
    for step,(batch_x, batch_y) in enumerate(train_loader):
        output = cnn(batch_x)
        loss = loss_func(output, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step % 50 == 0:
            test_output = cnn(test_x)
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == test_y.data.numpy()).astype(int).sum())/float(test_y.size(0))
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)

# print 10 predictions from test data
test_output = cnn(test_x[:10])
pred_y = torch.max(test_output, 1)[1].data.numpy()
print(pred_y, 'prediction number')
print(test_y[:10].numpy(), 'real number')

Epoch:  0 | train loss: 2.3003 | test accuracy: 0.11
Epoch:  0 | train loss: 0.6563 | test accuracy: 0.79
Epoch:  0 | train loss: 0.1750 | test accuracy: 0.89
Epoch:  0 | train loss: 0.1731 | test accuracy: 0.92
Epoch:  0 | train loss: 0.2425 | test accuracy: 0.93
Epoch:  0 | train loss: 0.1646 | test accuracy: 0.93
Epoch:  0 | train loss: 0.2048 | test accuracy: 0.94
Epoch:  0 | train loss: 0.1296 | test accuracy: 0.95
Epoch:  0 | train loss: 0.1420 | test accuracy: 0.96
Epoch:  0 | train loss: 0.0666 | test accuracy: 0.96
Epoch:  0 | train loss: 0.0721 | test accuracy: 0.97
Epoch:  0 | train loss: 0.0556 | test accuracy: 0.97
Epoch:  0 | train loss: 0.1138 | test accuracy: 0.96
Epoch:  0 | train loss: 0.0332 | test accuracy: 0.97
Epoch:  0 | train loss: 0.0246 | test accuracy: 0.97
Epoch:  0 | train loss: 0.1777 | test accuracy: 0.97
Epoch:  0 | train loss: 0.0350 | test accuracy: 0.98
Epoch:  0 | train loss: 0.0644 | test accuracy: 0.98
Epoch:  0 | train loss: 0.0285 | test accuracy